In [1]:
import os
from pathlib import Path

import json
import pandas as pd
from rdkit import Chem

cwd = os.getcwd()
if cwd.endswith('notebook/PG'):
    os.chdir('../..')
    cwd = os.getcwd()

from src.pg_modelling.ligand_utils import (
    generate_ccd_from_smiles, 
    generate_ccd_from_mol, 
    sanitize_ligand_name, 
    generate_conformation,
)
from src.pg_modelling.af3.scoring import process_af3_ligand_pulldown_results
from src.pg_modelling.protenix.scoring import process_protenix_ligand_pulldown_results

In [2]:
data_folder = Path(os.path.expanduser('~')) / 'Documents' / 'PG_modelling'
assert data_folder.is_dir()

In [3]:
glycan_modifications = pd.read_csv(
    data_folder / 'glycan_modifications' / 'glycan_modifications.csv', 
    index_col='ligand_name',
)
print(f'Number of fragments in subset: {len(glycan_modifications):,}')
glycan_modifications

Number of fragments in subset: 9


,Name,Synonym,Formula,Monoisotopic Mass,Modifications,Degree Amidation,Degree Acetylation,Ontology,PGN Units,Glycan Units,...,Peptide,SMILES,INCHIKEY,clogP,RT,[M+H]+,[M+Na]+,[M+K]+,[M+2H]2+,[M+3H]3+
ligand_name,,,,,,,,,,,,,,,,,,,,,
G-NAM,(G)(NAM)-None,GlcN.MurNAc--None,C17H30N2O12,454.17987,Amidase,0,1,G2,1,2,...,NaN,CC(=O)NC1C(O)OC(CO)C(OC2OC(CO)C(O)C(O)C2N)C1OC...,CCRWBXATDKHQDG-UHFFFAOYSA-N,-4.7896,0,455.18715,477.16909,493.14303,228.09722,152.40057
G-DAM,(G)(DAM)-None,GlcN.MurNAcOAc--None,C19H32N2O13,496.19044,Amidase,0,2,G2,1,2,...,NaN,CC(=O)NC1C(O)OC(COC(C)=O)C(OC2OC(CO)C(O)C(O)C2...,WMTUYCBSSKTNNR-UHFFFAOYSA-N,-4.2188,0,497.19772,519.17966,535.15360,249.10250,166.40409
NAG-MUR,(NAG)(MUR)-None,GlcNAc.MurN--None,C17H30N2O12,454.17987,Amidase,0,1,G2,1,2,...,NaN,CC(=O)NC1C(OC2C(CO)OC(O)C(N)C2OC(C)C(=O)O)OC(C...,IGYLVGQPIFKPAO-UHFFFAOYSA-N,-4.7896,0,455.18715,477.16909,493.14303,228.09722,152.40057
DAG-MUR,(DAG)(MUR)-None,GlcNAcOAc.MurN--None,C19H32N2O13,496.19044,Amidase,0,2,G2,1,2,...,NaN,CC(=O)NC1C(OC2C(CO)OC(O)C(N)C2OC(C)C(=O)O)OC(C...,ZSDDRWVIQWSQPJ-UHFFFAOYSA-N,-4.2188,0,497.19772,519.17966,535.15360,249.10250,166.40409
NAG-NAM,(NAG)(NAM)-None,GlcNAc.MurNAc--None,C19H32N2O13,496.19044,Amidase,0,2,G2,1,2,...,NaN,CC(=O)NC1C(OC2C(CO)OC(O)C(NC(C)=O)C2OC(C)C(=O)...,WQKIVDUWFNRJHE-UHFFFAOYSA-N,-4.6122,0,497.19772,519.17966,535.15360,249.10250,166.40409
DAG-NAM,(DAG)(NAM)-None,GlcNAcOAc.MurNAc--None,C21H34N2O14,538.20100,Amidase,0,3,G2,1,2,...,NaN,CC(=O)NC1C(OC2C(CO)OC(O)C(NC(C)=O)C2OC(C)C(=O)...,SUTALUMKMRSKIF-UHFFFAOYSA-N,-4.0414,0,539.20828,561.19022,577.16416,270.10778,180.40761
NAG-DAM,(NAG)(DAM)-None,GlcNAc.MurNAcOAc--None,C21H34N2O14,538.20100,Amidase,0,3,G2,1,2,...,NaN,CC(=O)NC1C(OC2C(COC(C)=O)OC(O)C(NC(C)=O)C2OC(C...,GKOAKWSMCSQKMH-UHFFFAOYSA-N,-4.0414,0,539.20828,561.19022,577.16416,270.10778,180.40761
DAG-DAM,(DAG)(DAM)-None,GlcNAcOAc.MurNAcOAc--None,C23H36N2O15,580.21157,Amidase,0,4,G2,1,2,...,NaN,CC(=O)NC1C(OC2C(COC(C)=O)OC(O)C(NC(C)=O)C2OC(C...,UGHTWFWJTFCKTB-UHFFFAOYSA-N,-3.4706,0,581.21885,603.20079,619.17473,291.11307,194.41114
G-MUR,(G)(MUR)-None,GlcN.MurN--None,C15H28N2O11,412.16931,Amidase,0,0,G2,1,2,...,NaN,CC(OC1C(N)C(O)OC(CO)C1OC1OC(CO)C(O)C(O)C1N)C(=O)O,UBADYJPVCNDSPI-UHFFFAOYSA-N,-4.9670,0,413.17659,435.15853,451.13247,207.09194,138.39705


In [5]:
ligand_folder = data_folder / 'glycan_modifications' / 'pdb'
ligand_folder.mkdir(exist_ok=True)

for ligand_name, row in glycan_modifications.iterrows():
    raw_name = row['Name']
    smiles = row['SMILES']

    mol = Chem.MolFromSmiles(smiles)
    try:
        mol = generate_conformation(mol)
    except ValueError:
        print(f'Error for ligand: {raw_name}')
        raise

    output_path = ligand_folder / f'{ligand_name}.pdb'
    Chem.MolToPDBFile(mol, output_path.as_posix())

[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling 

## Protenix results

In [8]:
protenix_lysozyme_folder = data_folder / 'glycan_modifications' / 'protenix__lysozyme_pulldown'
protenix_results_df = process_protenix_ligand_pulldown_results('Human_Lysozyme', protenix_lysozyme_folder, run_posebusters=True)
protenix_results_df

folder  \
protein_name   ligand_name                                     
Human_Lysozyme G-MUR          Human_Lysozyme_C_P61626__G-MUR   
               NAG-MUR      Human_Lysozyme_C_P61626__NAG-MUR   
               DAG-MUR      Human_Lysozyme_C_P61626__DAG-MUR   
               NAG-DAM      Human_Lysozyme_C_P61626__NAG-DAM   
               DAG-DAM      Human_Lysozyme_C_P61626__DAG-DAM   
               G-NAM          Human_Lysozyme_C_P61626__G-NAM   
               DAG-NAM      Human_Lysozyme_C_P61626__DAG-NAM   
               NAG-NAM      Human_Lysozyme_C_P61626__NAG-NAM   
               G-DAM          Human_Lysozyme_C_P61626__G-DAM   

                                                               structure_file  \
protein_name   ligand_name                                                      
Human_Lysozyme G-MUR        /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               NAG-MUR      /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               DAG-MUR      /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               NAG-DAM      /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               DAG-DAM      /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               G-NAM        /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               DAG-NAM      /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               NAG-NAM      /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               G-DAM        /Users/rs1521/Documents/PG_modelling/glycan_mo...   

                                seed       ptm      iptm  confidence  \
protein_name   ligand_name                                             
Human_Lysozyme G-MUR        seed_998  0.954696  0.986968    0.980513   
               NAG-MUR      seed_201  0.958441  0.979361    0.975177   
               DAG-MUR      seed_201  0.959054  0.979086    0.975080   
               NAG-DAM      seed_201  0.958758  0.975488    0.972142   
               DAG-DAM      seed_862  0.959768  0.974485    0.971542   
               G-NAM        seed_998  0.963097  0.971376    0.969720   
               DAG-NAM      seed_998  0.955708  0.967946    0.965498   
               NAG-NAM      seed_862  0.960965  0.965639    0.964704   
               G-DAM        seed_862  0.965220  0.978546    0.975880   

                            posebusters_score           posebusters_errors  
protein_name   ligand_name                                                  
Human_Lysozyme G-MUR                       20                         None  
               NAG-MUR                     20                         None  
               DAG-MUR                     20                         None  
               NAG-DAM                     20                         None  
               DAG-DAM                     20                         None  
               G-NAM                       20                         None  
               DAG-NAM                     20                         None  
               NAG-NAM                     20                         None  
               G-DAM                       19  minimum_distance_to_protein

## AF3

In [ ]:
ligand_folder_af3 = data_folder / 'glycan_modifications' / 'af3__inputs'
ligand_folder_af3.mkdir(exist_ok=True)

for ligand_name, row in glycan_modifications.iterrows():
    raw_name = row['Name']
    smiles = row['SMILES']

    ccd_code = f'LIG-PG-{ligand_name}'.replace('_', '-')
    ligand_seq = {
        'ligand': {
            'id': 'B',
            'ccdCodes': [ccd_code],
        }
    }
    try:
        ccd_data = generate_ccd_from_smiles(smiles, ccd_code)
    except ValueError:
        print(f'Error for ligand: {raw_name}')
        raise

    data = {
        'name': ligand_name,
        'sequences': [ligand_seq],
        'userCCD': ccd_data,
        'dialect': 'alphafold3',
        'version': 1,
    }
    with (ligand_folder_af3 / f'{ligand_name}.json').open('w') as f_out:
        json.dump(
            data, 
            f_out,
            indent=True,
        )

[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling 

### AF3 results

In [4]:
af3_output_folder = data_folder / 'glycan_modifications' / 'af3__lysozyme_pulldown'

results_df = process_af3_ligand_pulldown_results('Human_Lysozyme', af3_output_folder, run_posebusters=True)
results_df

folder  \
protein_name   ligand_name                                      
Human_Lysozyme LIG-PG-DAG-MUR  human_lysozyme_p61626__dag-mur   
               LIG-PG-NAG-MUR  human_lysozyme_p61626__nag-mur   
               LIG-PG-G-DAM      human_lysozyme_p61626__g-dam   
               LIG-PG-G-NAM      human_lysozyme_p61626__g-nam   
               LIG-PG-NAG-DAM  human_lysozyme_p61626__nag-dam   
               LIG-PG-DAG-DAM  human_lysozyme_p61626__dag-dam   
               LIG-PG-G-MUR      human_lysozyme_p61626__g-mur   
               LIG-PG-DAG-NAM  human_lysozyme_p61626__dag-nam   
               LIG-PG-NAG-NAM  human_lysozyme_p61626__nag-nam   

                                                                  structure_file  \
protein_name   ligand_name                                                         
Human_Lysozyme LIG-PG-DAG-MUR  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-NAG-MUR  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-G-DAM    /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-G-NAM    /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-NAG-DAM  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-DAG-DAM  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-G-MUR    /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-DAG-NAM  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-NAG-NAM  /Users/rs1521/Documents/PG_modelling/glycan_mo...   

                                ptm  iptm  confidence  posebusters_score  \
protein_name   ligand_name                                                 
Human_Lysozyme LIG-PG-DAG-MUR  0.87  0.92       0.910                 20   
               LIG-PG-NAG-MUR  0.86  0.91       0.900                 20   
               LIG-PG-G-DAM    0.86  0.88       0.876                 20   
               LIG-PG-G-NAM    0.85  0.86       0.858                 20   
               LIG-PG-NAG-DAM  0.85  0.85       0.850                 20   
               LIG-PG-DAG-DAM  0.84  0.80       0.808                 20   
               LIG-PG-G-MUR    0.84  0.79       0.800                 20   
               LIG-PG-DAG-NAM  0.84  0.80       0.808                 19   
               LIG-PG-NAG-NAM  0.83  0.75       0.766                 19   

                              posebusters_errors  
protein_name   ligand_name                        
Human_Lysozyme LIG-PG-DAG-MUR               None  
               LIG-PG-NAG-MUR               None  
               LIG-PG-G-DAM                 None  
               LIG-PG-G-NAM                 None  
               LIG-PG-NAG-DAM               None  
               LIG-PG-DAG-DAM               None  
               LIG-PG-G-MUR                 None  
               LIG-PG-DAG-NAM    internal_energy  
               LIG-PG-NAG-NAM    internal_energy

# Glycans + stem

In [43]:
glycans_with_stem = pd.read_csv(data_folder / 'glycan_modifications' / 'glycans_with_stem.csv')
print(f'Number of fragments in subset: {len(glycans_with_stem):,}')
if 'ligand_name' not in glycans_with_stem.columns:
    glycans_with_stem['ligand_name'] = glycans_with_stem['Name'].apply(sanitize_ligand_name)
    glycans_with_stem.to_csv(data_folder / 'glycan_modifications' / 'glycans_with_stem.csv', index=False)

glycans_with_stem = glycans_with_stem.set_index('ligand_name')
glycans_with_stem

Number of fragments in subset: 9


,Name,Synonym,Formula,Monoisotopic Mass,Modifications,Degree Amidation,Degree Acetylation,Ontology,PGN Units,Glycan Units,...,Peptide,SMILES,INCHIKEY,clogP,RT,[M+H]+,[M+Na]+,[M+K]+,[M+2H]2+,[M+3H]3+
ligand_name,,,,,,,,,,,,,,,,,,,,,
G-NAM-AqKAA,(G)(NAM)-AqKAA,GlcN.MurNAc--Ala.γ-isoGln.Lys.Ala.Ala,C37H65N9O18,923.44476,NaN,1,1,G2--P5,1,2,...,AqKAA,CC(=O)NC1C(O)OC(CO)C(OC2OC(CO)C(O)C(O)C2N)C1OC...,CEWPULLDOHGJFJ-UHFFFAOYSA-N,-7.9114,0,924.45204,946.43398,962.40792,462.72966,308.82220
G-DAM-AqKAA,(G)(DAM)-AqKAA,GlcN.MurNAcOAc--Ala.γ-isoGln.Lys.Ala.Ala,C39H67N9O19,965.45532,NaN,1,2,G2--P5,1,2,...,AqKAA,CC(=O)NC1C(O)OC(COC(C)=O)C(OC2OC(CO)C(O)C(O)C2...,MBBOMWPDFVBEFO-UHFFFAOYSA-N,-7.3406,0,966.46260,988.44454,1004.41848,483.73494,322.82572
NAG-MUR-AqKAA,(NAG)(MUR)-AqKAA,GlcNAc.MurN--Ala.γ-isoGln.Lys.Ala.Ala,C37H65N9O18,923.44476,NaN,1,1,G2--P5,1,2,...,AqKAA,CC(=O)NC1C(OC2C(CO)OC(O)C(N)C2OC(C)C(=O)NC(C)C...,XHMRJLWLLVNRID-UHFFFAOYSA-N,-7.9114,0,924.45204,946.43398,962.40792,462.72966,308.82220
DAG-MUR-AqKAA,(DAG)(MUR)-AqKAA,GlcNAcOAc.MurN--Ala.γ-isoGln.Lys.Ala.Ala,C39H67N9O19,965.45532,NaN,1,2,G2--P5,1,2,...,AqKAA,CC(=O)NC1C(OC2C(CO)OC(O)C(N)C2OC(C)C(=O)NC(C)C...,BOQLSOQRBUSEDT-UHFFFAOYSA-N,-7.3406,0,966.46260,988.44454,1004.41848,483.73494,322.82572
NAG-NAM-AqKAA,(NAG)(NAM)-AqKAA,GlcNAc.MurNAc--Ala.γ-isoGln.Lys.Ala.Ala,C39H67N9O19,965.45532,NaN,1,2,G2--P5,1,2,...,AqKAA,CC(=O)NC1C(OC2C(CO)OC(O)C(NC(C)=O)C2OC(C)C(=O)...,MYHLLWFYGIKMKM-UHFFFAOYSA-N,-7.7340,0,966.46260,988.44454,1004.41848,483.73494,322.82572
DAG-NAM-AqKAA,(DAG)(NAM)-AqKAA,GlcNAcOAc.MurNAc--Ala.γ-isoGln.Lys.Ala.Ala,C41H69N9O20,1007.46589,NaN,1,3,G2--P5,1,2,...,AqKAA,CC(=O)NC1C(OC2C(CO)OC(O)C(NC(C)=O)C2OC(C)C(=O)...,QTFTZFLPZOTKNK-UHFFFAOYSA-N,-7.1632,0,1008.47317,1030.45511,1046.42905,504.74023,336.82924
NAG-DAM-AqKAA,(NAG)(DAM)-AqKAA,GlcNAc.MurNAcOAc--Ala.γ-isoGln.Lys.Ala.Ala,C41H69N9O20,1007.46589,NaN,1,3,G2--P5,1,2,...,AqKAA,CC(=O)NC1C(OC2C(COC(C)=O)OC(O)C(NC(C)=O)C2OC(C...,ZXAGVKGKQYVLOI-UHFFFAOYSA-N,-7.1632,0,1008.47317,1030.45511,1046.42905,504.74023,336.82924
DAG-DAM-AqKAA,(DAG)(DAM)-AqKAA,GlcNAcOAc.MurNAcOAc--Ala.γ-isoGln.Lys.Ala.Ala,C43H71N9O21,1049.47645,NaN,1,4,G2--P5,1,2,...,AqKAA,CC(=O)NC1C(OC2C(COC(C)=O)OC(O)C(NC(C)=O)C2OC(C...,BMSUWBVYWHQPBP-UHFFFAOYSA-N,-6.5924,0,1050.48373,1072.46567,1088.43961,525.74551,350.83276
G-MUR-AqKAA,(G)(MUR)-AqKAA,GlcN.MurN--Ala.γ-isoGln.Lys.Ala.Ala,C35H63N9O17,881.43419,NaN,1,0,G2--P5,1,2,...,AqKAA,CC(NC(=O)C(C)NC(=O)C(CCCCN)NC(=O)CCC(NC(=O)C(C...,PKMDYYVCWWROLE-UHFFFAOYSA-N,-8.0888,0,882.44147,904.42341,920.39735,441.72438,294.81868


In [20]:
ligand_folder_with_stem_af3 = data_folder / 'glycan_modifications' / 'af3__glycans_with_stem'
ligand_folder_with_stem_af3.mkdir(exist_ok=True)

ligand_folder_with_stem_protenix = data_folder / 'glycan_modifications' / 'protenix__glycans_with_stem'
ligand_folder_with_stem_protenix.mkdir(exist_ok=True)

for ligand_name, row in glycans_with_stem.iterrows():
    raw_name = row['Name']
    smiles = row['SMILES']

    mol = Chem.MolFromSmiles(smiles)
    try:
        mol = generate_conformation(mol)
    except ValueError:
        print(f'Error for ligand: {raw_name}')
        raise

    output_path = ligand_folder_with_stem_protenix / f'{ligand_name}.pdb'
    Chem.MolToPDBFile(mol, output_path.as_posix())

    ccd_code = f'LIG-PG-{ligand_name}'.replace('_', '-')
    ligand_seq = {
        'ligand': {
            'id': 'B',
            'ccdCodes': [ccd_code],
        }
    }
    try:
        ccd_data = generate_ccd_from_mol(mol, ccd_code)
    except ValueError:
        print(f'Error for ligand: {raw_name}')
        raise

    data = {
        'name': ligand_name,
        'sequences': [ligand_seq],
        'userCCD': ccd_data,
        'dialect': 'alphafold3',
        'version': 1,
    }
    with (ligand_folder_with_stem_af3 / f'{ligand_name}.json').open('w') as f_out:
        json.dump(
            data, 
            f_out,
            indent=True,
        )

[23:25:25] Molecule does not have explicit Hs. Consider calling AddHs()
[23:25:25] Molecule does not have explicit Hs. Consider calling AddHs()
[23:25:25] Molecule does not have explicit Hs. Consider calling AddHs()
[23:25:25] Molecule does not have explicit Hs. Consider calling AddHs()
[23:25:25] Molecule does not have explicit Hs. Consider calling AddHs()
[23:25:25] Molecule does not have explicit Hs. Consider calling AddHs()
[23:25:25] Molecule does not have explicit Hs. Consider calling AddHs()
[23:25:25] Molecule does not have explicit Hs. Consider calling AddHs()
[23:25:25] Molecule does not have explicit Hs. Consider calling AddHs()
[23:25:25] Molecule does not have explicit Hs. Consider calling AddHs()
[23:25:25] Molecule does not have explicit Hs. Consider calling AddHs()
[23:25:25] Molecule does not have explicit Hs. Consider calling AddHs()
[23:25:25] Molecule does not have explicit Hs. Consider calling AddHs()
[23:25:25] Molecule does not have explicit Hs. Consider calling 

### Results AF3

In [3]:
af3_output_folder = data_folder / 'glycan_modifications' / 'af3__lysozyme_pulldown_with_stem'

results_df = process_af3_ligand_pulldown_results('Human_Lysozyme', af3_output_folder, run_posebusters=True)
results_df

folder  \
protein_name   ligand_name                                                  
Human_Lysozyme LIG-PG-NAG-NAM-AqKAA  human_lysozyme_p61626__nag-nam-aqkaa   
               LIG-PG-NAG-DAM-AqKAA  human_lysozyme_p61626__nag-dam-aqkaa   
               LIG-PG-DAG-MUR-AqKAA  human_lysozyme_p61626__dag-mur-aqkaa   
               LIG-PG-G-NAM-AqKAA      human_lysozyme_p61626__g-nam-aqkaa   
               LIG-PG-G-DAM-AqKAA      human_lysozyme_p61626__g-dam-aqkaa   
               LIG-PG-G-MUR-AqKAA      human_lysozyme_p61626__g-mur-aqkaa   
               LIG-PG-DAG-NAM-AqKAA  human_lysozyme_p61626__dag-nam-aqkaa   
               LIG-PG-DAG-DAM-AqKAA  human_lysozyme_p61626__dag-dam-aqkaa   
               LIG-PG-NAG-MUR-AqKAA  human_lysozyme_p61626__nag-mur-aqkaa   

                                                                        structure_file  \
protein_name   ligand_name                                                               
Human_Lysozyme LIG-PG-NAG-NAM-AqKAA  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-NAG-DAM-AqKAA  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-DAG-MUR-AqKAA  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-G-NAM-AqKAA    /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-G-DAM-AqKAA    /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-G-MUR-AqKAA    /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-DAG-NAM-AqKAA  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-DAG-DAM-AqKAA  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-NAG-MUR-AqKAA  /Users/rs1521/Documents/PG_modelling/glycan_mo...   

                                      ptm  iptm  confidence  \
protein_name   ligand_name                                    
Human_Lysozyme LIG-PG-NAG-NAM-AqKAA  0.82  0.75       0.764   
               LIG-PG-NAG-DAM-AqKAA  0.80  0.71       0.728   
               LIG-PG-DAG-MUR-AqKAA  0.79  0.65       0.678   
               LIG-PG-G-NAM-AqKAA    0.83  0.80       0.806   
               LIG-PG-G-DAM-AqKAA    0.82  0.77       0.780   
               LIG-PG-G-MUR-AqKAA    0.82  0.73       0.748   
               LIG-PG-DAG-NAM-AqKAA  0.81  0.73       0.746   
               LIG-PG-DAG-DAM-AqKAA  0.79  0.67       0.694   
               LIG-PG-NAG-MUR-AqKAA  0.79  0.63       0.662   

                                     posebusters_score  \
protein_name   ligand_name                               
Human_Lysozyme LIG-PG-NAG-NAM-AqKAA                 20   
               LIG-PG-NAG-DAM-AqKAA                 20   
               LIG-PG-DAG-MUR-AqKAA                 20   
               LIG-PG-G-NAM-AqKAA                   19   
               LIG-PG-G-DAM-AqKAA                   19   
               LIG-PG-G-MUR-AqKAA                   19   
               LIG-PG-DAG-NAM-AqKAA                 19   
               LIG-PG-DAG-DAM-AqKAA                 19   
               LIG-PG-NAG-MUR-AqKAA                 19   

                                              posebusters_errors  
protein_name   ligand_name                                        
Human_Lysozyme LIG-PG-NAG-NAM-AqKAA                         None  
               LIG-PG-NAG-DAM-AqKAA                         None  
               LIG-PG-DAG-MUR-AqKAA                         None  
               LIG-PG-G-NAM-AqKAA    minimum_distance_to_protein  
               LIG-PG-G-DAM-AqKAA    minimum_distance_to_protein  
               LIG-PG-G-MUR-AqKAA    minimum_distance_to_protein  
               LIG-PG-DAG-NAM-AqKAA  minimum_distance_to_protein  
               LIG-PG-DAG-DAM-AqKAA  minimum_distance_to_protein  
               LIG-PG-NAG-MUR-AqKAA  minimum_distance_to_protein

In [4]:
af3_output_folder = data_folder / 'glycan_modifications' / 'af3__lysozyme_pulldown_with_stem'
results_all = process_af3_ligand_pulldown_results('Human_Lysozyme', af3_output_folder, run_posebusters=True, score_all_sample=True)
results_all

structure_file  \
protein_name   ligand_name                                                               
Human_Lysozyme LIG-PG-G-NAM-AqKAA    /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-G-DAM-AqKAA    /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-NAG-NAM-AqKAA  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-DAG-NAM-AqKAA  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-G-MUR-AqKAA    /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-NAG-DAM-AqKAA  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-DAG-MUR-AqKAA  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-DAG-DAM-AqKAA  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-NAG-MUR-AqKAA  /Users/rs1521/Documents/PG_modelling/glycan_mo...   

                                      ptm  iptm  confidence  \
protein_name   ligand_name                                    
Human_Lysozyme LIG-PG-G-NAM-AqKAA    0.83  0.79       0.798   
               LIG-PG-G-DAM-AqKAA    0.82  0.76       0.772   
               LIG-PG-NAG-NAM-AqKAA  0.82  0.75       0.764   
               LIG-PG-DAG-NAM-AqKAA  0.81  0.72       0.738   
               LIG-PG-G-MUR-AqKAA    0.81  0.71       0.730   
               LIG-PG-NAG-DAM-AqKAA  0.80  0.71       0.728   
               LIG-PG-DAG-MUR-AqKAA  0.79  0.65       0.678   
               LIG-PG-DAG-DAM-AqKAA  0.78  0.64       0.668   
               LIG-PG-NAG-MUR-AqKAA  0.76  0.56       0.600   

                                     posebusters_score posebusters_errors  
protein_name   ligand_name                                                 
Human_Lysozyme LIG-PG-G-NAM-AqKAA                   20               None  
               LIG-PG-G-DAM-AqKAA                   20               None  
               LIG-PG-NAG-NAM-AqKAA                 20               None  
               LIG-PG-DAG-NAM-AqKAA                 20               None  
               LIG-PG-G-MUR-AqKAA                   20               None  
               LIG-PG-NAG-DAM-AqKAA                 20               None  
               LIG-PG-DAG-MUR-AqKAA                 20               None  
               LIG-PG-DAG-DAM-AqKAA                 20               None  
               LIG-PG-NAG-MUR-AqKAA                 20               None

In [8]:
results_all.loc[('Human_Lysozyme', 'LIG-PG-NAG-NAM-AqKAA'), 'structure_file']

'/Users/rs1521/Documents/PG_modelling/glycan_modifications/af3__lysozyme_pulldown_with_stem/human_lysozyme_p61626__nag-nam-aqkaa/seed-52_sample-0/model.cif'

In [9]:
results_all.loc[('Human_Lysozyme', 'LIG-PG-NAG-MUR-AqKAA'), 'structure_file']

'/Users/rs1521/Documents/PG_modelling/glycan_modifications/af3__lysozyme_pulldown_with_stem/human_lysozyme_p61626__nag-mur-aqkaa/seed-49_sample-2/model.cif'

### Results Protenix

In [ ]:
protenix_lysozyme_folder_with_stem = data_folder / 'glycan_modifications' / 'protenix__lysozyme_pulldown_with_stem'
protenix_results_df = process_protenix_ligand_pulldown_results('Human_Lysozyme', protenix_lysozyme_folder_with_stem, run_posebusters=True)
protenix_results_df

structure_file  \
protein_name   ligand_name                                                        
Human_Lysozyme DAG-MUR-AqKAA  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               G-DAM-AqKAA    /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               NAG-NAM-AqKAA  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               G-NAM-AqKAA    /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               DAG-NAM-AqKAA  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               G-MUR-AqKAA    /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               NAG-DAM-AqKAA  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               NAG-MUR-AqKAA  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               DAG-DAM-AqKAA  /Users/rs1521/Documents/PG_modelling/glycan_mo...   

                                   ptm      iptm  confidence  \
protein_name   ligand_name                                     
Human_Lysozyme DAG-MUR-AqKAA  0.959234  0.966989    0.965438   
               G-DAM-AqKAA    0.970330  0.979125    0.977366   
               NAG-NAM-AqKAA  0.957819  0.974664    0.971295   
               G-NAM-AqKAA    0.966699  0.982339    0.979211   
               DAG-NAM-AqKAA  0.955281  0.961944    0.960612   
               G-MUR-AqKAA    0.969554  0.976267    0.974924   
               NAG-DAM-AqKAA  0.959189  0.970592    0.968311   
               NAG-MUR-AqKAA  0.958705  0.967770    0.965957   
               DAG-DAM-AqKAA  0.955395  0.959657    0.958805   

                              posebusters_score           posebusters_errors  \
protein_name   ligand_name                                                     
Human_Lysozyme DAG-MUR-AqKAA                 20                         None   
               G-DAM-AqKAA                   20                         None   
               NAG-NAM-AqKAA                 20                         None   
               G-NAM-AqKAA                   20                         None   
               DAG-NAM-AqKAA                 20                         None   
               G-MUR-AqKAA                   20                         None   
               NAG-DAM-AqKAA                 20                         None   
               NAG-MUR-AqKAA                 20                         None   
               DAG-DAM-AqKAA                 19  minimum_distance_to_protein   

                              energy_ratio  
protein_name   ligand_name                  
Human_Lysozyme DAG-MUR-AqKAA         8.128  
               G-DAM-AqKAA           9.761  
               NAG-NAM-AqKAA        10.121  
               G-NAM-AqKAA          10.155  
               DAG-NAM-AqKAA        11.614  
               G-MUR-AqKAA          12.921  
               NAG-DAM-AqKAA        13.881  
               NAG-MUR-AqKAA        58.011  
               DAG-DAM-AqKAA        12.929

In [4]:
protenix_results_df.loc[('Human_Lysozyme', 'NAG-NAM-AqKAA'), 'structure_file']

'/Users/rs1521/Documents/PG_modelling/glycan_modifications/protenix__lysozyme_pulldown_with_stem/Human_Lysozyme_C_P61626__NAG-NAM-AqKAA/seed_732/predictions/Human_Lysozyme_C_P61626__NAG-NAM-AqKAA_seed_732_sample_0.cif'

In [5]:
protenix_results_df.loc[('Human_Lysozyme', 'NAG-MUR-AqKAA'), 'structure_file']

'/Users/rs1521/Documents/PG_modelling/glycan_modifications/protenix__lysozyme_pulldown_with_stem/Human_Lysozyme_C_P61626__NAG-MUR-AqKAA/seed_735/predictions/Human_Lysozyme_C_P61626__NAG-MUR-AqKAA_seed_735_sample_0.cif'

## PLIP

[Protein-Ligand Interaction Profiler (PLIP)](https://github.com/pharmai/plip)

In [20]:
import subprocess
import tempfile

with tempfile.TemporaryDirectory() as tmpdir:
    pdb_path = Path(tmpdir) / 'structure.pdb'
    cif_path = protenix_results_df.loc[('Human_Lysozyme', 'NAG-NAM-AqKAA'), 'structure_file']

    subprocess.run(['obabel', '-icif', cif_path, '-opdb', '-O', pdb_path.as_posix()])

    subprocess.run(['plip', '-f', pdb_path.as_posix(), '-o', tmpdir, '-t'])

    with (Path(tmpdir) / 'report.txt').open() as f:
        content = f.read()
        print(content)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is Human_Lysozyme_C_P61626__NAG-NAM-AqKAA_seed_732_sample_0_predicted_by_protenix)

1 molecule converted


None


2025-02-05 23:49:59,330 [INFO] [plipcmd.py:134] plip.plipcmd: Protein-Ligand Interaction Profiler (PLIP) 2.4.0
2025-02-05 23:49:59,330 [INFO] [plipcmd.py:135] plip.plipcmd: brought to you by: PharmAI GmbH (2020-2021) - www.pharm.ai - hello@pharm.ai
2025-02-05 23:49:59,330 [INFO] [plipcmd.py:136] plip.plipcmd: please cite: Adasme,M. et al. PLIP 2021: expanding the scope of the protein-ligand interaction profiler to DNA and RNA. Nucl. Acids Res. (05 May 2021), gkab294. doi: 10.1093/nar/gkab294
2025-02-05 23:49:59,330 [INFO] [plipcmd.py:60] plip.plipcmd: starting analysis of structure.pdb


Prediction of noncovalent interactions for PDB structure STRUCTURE_PROTEIN
Created on 2025/02/05 using PLIP v2.4.0

If you are using PLIP in your work, please cite:
Adasme,M. et al. PLIP 2021: expanding the scope of the protein-ligand interaction profiler to DNA and RNA. Nucl. Acids Res. (05 May 2021), gkab294. doi: 10.1093/nar/gkab294
Analysis was done on model 1.

l01:B:1 (l01) - SMALLMOLECULE
-----------------------------
Interacting chain(s): A


**Hydrophobic Interactions**
+-------+---------+----------+-----------+-------------+--------------+------+--------------+---------------+-----------------------+-----------------------+
| RESNR | RESTYPE | RESCHAIN | RESNR_LIG | RESTYPE_LIG | RESCHAIN_LIG | DIST | LIGCARBONIDX | PROTCARBONIDX | LIGCOO                | PROTCOO               | 
+=======+=========+==========+===========+=============+==============+======+==============+===============+=======================+=======================+
| 81    | TYR     | A        | 1         

2025-02-05 23:49:59,547 [INFO] [plipcmd.py:175] plip.plipcmd: finished analysis, find the result files in /var/folders/xd/714yy8tn5vz90_wtnn7vglhc0000gq/T/tmp5ob05_eq/


## PoseBusters full output

In [4]:
from src.pg_modelling.ligand_utils import run_pose_busters

nag_nam_posebusters = run_pose_busters(
    #Path(protenix_results_df.loc[('Human_Lysozyme', 'NAG-NAM-AqKAA'), 'structure_file']),
    Path(
        data_folder / 
        'glycan_modifications/protenix__lysozyme_pulldown_with_stem/Human_Lysozyme_C_P61626__NAG-NAM-AqKAA/seed_732/predictions/Human_Lysozyme_C_P61626__NAG-NAM-AqKAA_seed_732_sample_0.cif'
    ),
    'l01',
    full_report=True,
)
nag_mur_posebusters = run_pose_busters(
    #Path(protenix_results_df.loc[('Human_Lysozyme', 'NAG-MUR-AqKAA'), 'structure_file']),
    Path(
        data_folder / 
        'glycan_modifications/protenix__lysozyme_pulldown_with_stem/Human_Lysozyme_C_P61626__NAG-MUR-AqKAA/seed_735/predictions/Human_Lysozyme_C_P61626__NAG-MUR-AqKAA_seed_735_sample_0.cif'
    ),
    'l01',
    full_report=True,
)

In [5]:
for col in nag_nam_posebusters.columns:
    if 'energy' in col:
        print(col, nag_nam_posebusters.iloc[0][col])

internal_energy True
ensemble_avg_energy 181.25337483362702
mol_pred_energy 1834.4677072341537
energy_ratio 10.121012692414785


In [6]:
for col in nag_mur_posebusters.columns:
    if 'energy' in col:
        print(col, nag_mur_posebusters.iloc[0][col])

internal_energy True
ensemble_avg_energy 180.21084748875165
mol_pred_energy 10454.166811167526
energy_ratio 58.01075216529377
